# Analiza tu música de Spotify
A continuación podrás ver un ejemplo de cómo se puede utilizar la ciencia de datos para conocer tus gustos musicales, escpecíficamente para conocer que tienen de similar las canciones que escuchas. El objetivo de este taller es poder separar en grupos de canciones similares la música que tienes en tus me gusta, para finalmente analizar las características que tiene cada uno de estos grupos.


![Portada](https://entrackr.com/storage/2019/01/spotify-.jpg)

## Extracción de Datos

En la primera celda se importan las librerías que se utilizarán para todo el proceso.

In [ ]:
#Conexión a Spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

#Manipulación de Datos
import pandas as pd
import numpy as np

#Limpieza de Datos
import re
from googletrans import Translator

#Maachine Learning
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#Visualización de Datos
import matplotlib.pyplot as plt

En la siguiente celda se realiza la conexión a Spotify, donde se abrirá una pestaña para que inicies sesión con tu cuenta.

<img src='https://d500.epimg.net/cincodias/imagenes/2022/02/11/lifestyle/1644599664_740393_1644599921_noticia_normal.jpg' width=300 align='left'>

In [ ]:
username = input("Ingrese su usuario: ")
client_id = ""
client_secret = ""
redirect_uri = "http://localhost:7777/callback"
scope = 'user-library-read playlist-modify-private playlist-modify-public'

auth = SpotifyOAuth(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth_manager=auth)
user = sp.current_user()
print("Bienvenido " + user['display_name'])

Ahora vamos a extraer las canciones que tienes guardadas en tus me gusta.

<img src='https://imgmedia.larepublica.pe/1200x660/larepublica/original/2021/02/25/6038156cd9b64d607e178a18.jpg' width=300 align='left'>

In [ ]:
results = sp.current_user_saved_tracks(limit=50)
items = results['items']
tracks = [item['track'] for item in items]

uris = [track["uri"] for track in tracks]
titles = [track['name'] for track in tracks]
albums = [track['album']['name'] for track in tracks]
artists = [track['artists'][0]['name'] for track in tracks]
release_dates = [track['album']['release_date'] for track in tracks]
years = [date[:4] if date else None for date in release_dates]
popularities = [track['popularity'] for track in tracks]
audio_features = sp.audio_features(uris)

while results["next"]:
    results = sp.next(results)
    items = results['items']
    tracks = [item['track'] for item in items]
    uris += [track["uri"] for track in tracks]
    titles += [track['name'] for track in tracks]
    albums += [track['album']['name'] for track in tracks]
    artists += [track['artists'][0]['name'] for track in tracks]
    release_dates = [track['album']['release_date'] for track in tracks]
    years += [date[:4] if date else None for date in release_dates]
    popularities += [track['popularity'] for track in tracks]
    audio_features += sp.audio_features([track["uri"] for track in tracks])

print(len(uris), "canciones encontradas")

## Limpieza de Datos

Una vez tenemos las canciones, es necesario limpiar los datos para tenerlos de la forma que más nos conviene para el modelo. 

(Este paso puede tardar unos minutos)

<img src='https://datascientest.com/es/wp-content/uploads/sites/7/2021/05/illu_data_cleaning_blog_2-07.png' width=300 align='left'>

In [ ]:
print("Dependiendo de la cantidad de canciones, esto pude tardar unos minutos...")
df = pd.DataFrame(audio_features)
df['uri'] = uris
df['title'] = titles
df['album'] = albums
df['artist'] = artists
df['year'] = years
df['year'] = df['year'].astype(int)
df['popularity'] = popularities
detector = Translator()

def is_spanish(text):
    try:
        text = text.lower().replace('remix', '')
        text = re.sub(r'\([^)]*\)', '', text)
        return detector.detect(text).lang == 'es'
    except:
        return False
df['spanish'] = (df.title + ', ' + df.album).apply(is_spanish)
df = df[['uri', 'title', 'album', 'artist', 'spanish', 'year', 'tempo', 'energy',
         'danceability', 'loudness', 'mode', 'liveness', 'valence', 'duration_ms', 
         'acousticness', 'instrumentalness', 'speechiness', 'popularity']]
df.head()

## Modelo

<img src='https://ict.iitk.ac.in/wp-content/uploads/machine-learning-1.jpg' width=850>

En el primer paso elegimos qué variables vamos a tomar en cuenta para el modelo.

In [ ]:
vars_to_use = ['tempo', 'energy', 'danceability', 'loudness', 'mode', 'spanish',
               'valence', 'acousticness', 'speechiness']
df_clusters = df[vars_to_use]
scaler = StandardScaler()
df_clusters = scaler.fit_transform(df_clusters)

Luego probamos el modelo con diferentes cantidades de clusters. Esto nos sirve para saber cuál es la cantidad de grupos óptima para el problema. Entre más variados sean tus gustos musicales, es más probable que la cantidad óptima de clusters sea mayor.

In [ ]:
wcss = []
silhouette_scores = []

for k in range(2, 13):
    kmeans = KMeans(n_clusters=k, random_state=18)
    kmeans.fit(df_clusters)
    wcss.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(df_clusters, kmeans.labels_))

plt.plot(range(2, 13), wcss)
plt.title('Elbow Curve')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()

plt.plot(range(2, 13), silhouette_scores)
plt.title('Silhouette Score')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.show()

Es hora de correr el modelo final, veamos cuantas canciones pertenecen a cada uno de los clusters generados.

<img src='https://www.unica360.com/wp-content/uploads/segmentacion_dimensiones.jpg' align='left' width=300>

In [ ]:
k = int(input('Cantidad de clusters: '))
kmeans = KMeans(n_clusters=k, random_state=18)
clusters = kmeans.fit_predict(df_clusters)
df_results = df.copy()
df_results["cluster"] = clusters
df_results["cluster"].value_counts()

Por úlitmo guardamos los resultados en un archivo csv, el cual utilizaremos en PowerBI para analizar los resultados del modelo.

In [ ]:
df_results.to_csv("canciones.csv", index=False)

## Ponle nombre a los grupos

Luego de haber analizado los reultados, podemos ponerle un nombre a cada grupo de canciones.

In [ ]:
nombres_playlists = []
for i in range(k):
    nombres_playlists.append(input("Nombre del cluster " + str(i) + ": "))
for c, nombre in enumerate(nombres_playlists):
    playlist = sp.user_playlist_create(user['id'], nombre, public=True)
    tracks = df_results.uri[df_results.cluster == c].to_list()

    batches = [tracks[i:i+100] for i in range(0, len(tracks), 100)]

    for batch in batches:
        sp.playlist_add_items(playlist['uri'], batch)